In [1]:
! pip install ipykernel

In [2]:
# ===============================
# STEP 1 : Import Libraries
# ===============================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme()

In [3]:
df = pd.read_excel(r"C:\Users\mk744\Downloads\JPM25-26 TrackingSheetFinal (1).xlsx", sheet_name="Employees Enrollment Data")
df.columns

Index(['Timestamp', 'Status', 'Batch Code', 'First Name', 'Last Name',
       'Email ID', 'Full Name ', 'Duplicate', 'Contact Number', 'Gender',
       'DOB (Date Of Birth)', 'Marital Status', 'Religion', 'Category',
       'Parent/Guardian Name', 'Parent/Guardian Relation',
       'Parent's Occupation', 'Type of House', 'Household Income (Yearly)',
       'Current Address', 'Pincode ', 'City',
       'Type of Smart Devices Available', 'Highest Qualification', 'Stream',
       'Education Institute', 'Year Of Completion', 'Your Designation',
       'Employment Type', 'Company Name', 'Job Location', 'Job Location.1',
       'Are you currently working?',
       'ID Proof (Aadhaar Card / Pan Card / Voter ID / Office ID card / Any other ID Proof',
       'Education Document (Certificate / Marksheet of Highest Qualification)',
       'Photograph', 'Pin Code', 'Email Address', 'ID Proof Number',
       'Enquiry ID', 'Enrollment ID', ' ', 'Reasons for Non Enrolement',
       'Enrolled By', 'As

In [4]:
# Convert DOB → Age
df["Age"] = pd.to_datetime("today").year - pd.to_datetime(df["DOB (Date Of Birth)"]).dt.year

# Rename target column
df["Status"] = df["Status"].map({"Completed":1, "Dropped":0, "Fail":0}).fillna(0)
# ---- DROP DUPLICATES ----
df.drop_duplicates(
    subset = ["Email ID", "Enrollment ID", "Contact Number"],
    keep = "first",
    inplace = True
)

In [5]:
# ===============================
# STEP 3 : Select Features
# ===============================
features = df[[
    'First Name', 
    'Last Name',
    "Age",
    "Gender",
    "City",
    "Highest Qualification",
    "Stream",
    "Year Of Completion",
    "Are you currently working?",
    "Company Name",
    "Your Designation",
    "Employment Type"
]]

In [6]:
df_cols = features.rename(columns={"First Name": "First_Name", "Last Name": "Last_Name","Highest Qualification" : "Highest_Qualification", "Year Of Completion" : "Year_Of_Completion",
                         "Are you currently working?": "Are_you_currently_working", "Company Name": "Company_Name",
                         "Your Designation": "Your_Designation", "Employment Type": "Employment_Type"})
df_cols

,First_Name,Last_Name,Age,Gender,City,Highest_Qualification,Stream,Year_Of_Completion,Are_you_currently_working,Company_Name,Your_Designation,Employment_Type
0,Dimple,Nimesh,27,Female,faridabad,10th Pass,Arts,2018,No,Bharat Finance,Sales,Full Time
1,Khushbu,Yadav,20,Female,Delhi,12th Pass,Arts,2022,Yes,Bharat Finance,Executive,Full Time
2,Nisha,Jangid,20,Female,New delhi,12th Pass,Arts,2023,Yes,Bharat Finance,Executive,Full Time
3,Tara,Tara,24,Female,Delhi,12th Pass,Arts,2022,Yes,Bharat Finance,Executive,Full Time
4,Nitin,Chaurasiya,24,Male,New Delhi,Graduation,Science,2020,Yes,Bharat Finance,Executive,Full Time
...,...,...,...,...,...,...,...,...,...,...,...,...
146,Jeeva,Devendra,21,Male,Mumbai,12th Pass,Science,2025,No,Swiggy pvt.ltd,Delivery partner,Self Employed
147,Sania,Khan,20,Female,Mumbai,Graduation,Commerce,2025,Yes,Axis bank,Sale office,Full Time
148,Khan,Tarannum,22,Female,Mumbai,Graduation,Commerce,2024,No,Mustaq enterprises,Sales,Full Time
149,Vitthal,Jadhav,28,Male,Mumbai,Graduation,Science,2020,Yes,Harjai Technologies- Client: Essar Capital Ltd.,Data Entry Executive,Contract


In [7]:
df_cols.isnull().sum()

First_Name                   0
Last_Name                    0
Age                          0
Gender                       0
City                         0
Highest_Qualification        0
Stream                       0
Year_Of_Completion           0
Are_you_currently_working    0
Company_Name                 0
Your_Designation             0
Employment_Type              0
dtype: int64

In [8]:
df_cols['Company_Name'].value_counts().reset_index(name='count').sort_values(by='count', ascending=False)

,Company_Name,count
0,Dorset Industries Pvt Ltd,50
1,AMRIT CEMENT LTD,48
2,Hitachi cash management services,18
3,Bharat Finance,17
4,Factset,2
5,Startek,1
6,Shriram Finance limited,1
7,Infinix,1
8,Permeier engeries,1
9,Nirmaan organization,1


In [9]:
from tabulate import tabulate
print(tabulate(df_cols['Your_Designation'].value_counts().reset_index(name='count').sort_values(by='count', ascending=False), headers='keys', tablefmt='pretty'))

+-----+--------------------------------------+-------+
|     |           Your_Designation           | count |
+-----+--------------------------------------+-------+
|  0  |              Executive               |   9   |
|  1  |              Executive               |   7   |
|  2  |           Senior Executive           |   5   |
|  3  |                Sales                 |   3   |
|  4  |           Graduate Trainee           |   3   |
|  5  |               Manager                |   3   |
|  6  |                 MIS                  |   3   |
|  7  |        Deputy General Manager        |   3   |
|  8  |             Data Analyst             |   2   |
|  9  |           Sales executive            |   2   |
| 10  |          Executive Officer           |   2   |
| 11  |         HRMS IMPLEMENTATION.         |   2   |
| 12  |                 DME                  |   2   |
| 13  |          Process cordinator          |   2   |
| 14  |          Assistant manager           |   2   |
| 15  |   

In [10]:
df_cols.columns

Index(['First_Name', 'Last_Name', 'Age', 'Gender', 'City',
       'Highest_Qualification', 'Stream', 'Year_Of_Completion',
       'Are_you_currently_working', 'Company_Name', 'Your_Designation',
       'Employment_Type'],
      dtype='object')

In [11]:

def suitability_label(designation):
    if pd.isna(designation):
        return "Senior_Not_Suitable"
    
    d = designation.lower().strip()

    # ----------------------
    # 1️⃣ STRICT SENIOR ROLES
    # ----------------------
    senior_keywords = [
        "deputy general manager",
        "general manager",
        "chief manager",
        "sr chief manager",
        "agm",
        "dgm",
        "avp",
        "vice",
        "gm",
        "product manager",
        "dgm - production",
        "chief manager - sales digitization",
        "import manager",
        "deputy marketing manager",
        "sr manager",
        "senior manager",
        "sr. deputy manager",
        "deputy ea to md",
        "marketing team lead",
        "team lead",
        "cpo",
        "ea to md",
        "chief",
        "head",
        "jr. data analyst"
    ]

    # ----------------------
    # 2️⃣ MID SENIOR / SPECIALIST
    # ----------------------
    mid_senior_keywords = [
        "senior executive",
        "sr. executive",
        "sr executive",
        "assistant manager",
        "assistant manager hr",
        "senior officer",
        "sr officer",
        "officer",
        "hrms implementation",
        "research analyst",
        "senior data analyst",
        "account finance executive",
        "business associate",
        "sr assistant manager",
        "sr manager hr"
    ]

    # ----------------------
    # 3️⃣ BEST FIT (COURSE SUITABLE)
    # ----------------------
    suitable_keywords = [
        "graduate trainee",
        "junior executive",
        "junior",
        "executive",
        "data entry executive",
        "telecaller",
        "mis",
        "team member",
        "supervisor",
        "trainee",
        "help desk",
        "sales",
        "delivery partner"
    ]

    # Exact Priority Handling
    if any(k in d for k in senior_keywords):
        return "Senior_Not_Suitable"
    
    if any(k in d for k in mid_senior_keywords):
        return "MidSenior_Not_Suitable"
    
    if any(k in d for k in suitable_keywords):
        return "Suitable"

    # Default: assume NOT suitable
    return "Senior_Not_Suitable"


df_cols["Suitability_Label"] = df_cols["Your_Designation"].apply(suitability_label)

df_cols["Suitability_Label"].value_counts()


Suitability_Label
Senior_Not_Suitable       65
Suitable                  50
MidSenior_Not_Suitable    35
Name: count, dtype: int64

In [12]:
df_cols

,First_Name,Last_Name,Age,Gender,City,Highest_Qualification,Stream,Year_Of_Completion,Are_you_currently_working,Company_Name,Your_Designation,Employment_Type,Suitability_Label
0,Dimple,Nimesh,27,Female,faridabad,10th Pass,Arts,2018,No,Bharat Finance,Sales,Full Time,Suitable
1,Khushbu,Yadav,20,Female,Delhi,12th Pass,Arts,2022,Yes,Bharat Finance,Executive,Full Time,Suitable
2,Nisha,Jangid,20,Female,New delhi,12th Pass,Arts,2023,Yes,Bharat Finance,Executive,Full Time,Suitable
3,Tara,Tara,24,Female,Delhi,12th Pass,Arts,2022,Yes,Bharat Finance,Executive,Full Time,Suitable
4,Nitin,Chaurasiya,24,Male,New Delhi,Graduation,Science,2020,Yes,Bharat Finance,Executive,Full Time,Suitable
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Jeeva,Devendra,21,Male,Mumbai,12th Pass,Science,2025,No,Swiggy pvt.ltd,Delivery partner,Self Employed,Suitable
147,Sania,Khan,20,Female,Mumbai,Graduation,Commerce,2025,Yes,Axis bank,Sale office,Full Time,Senior_Not_Suitable
148,Khan,Tarannum,22,Female,Mumbai,Graduation,Commerce,2024,No,Mustaq enterprises,Sales,Full Time,Suitable
149,Vitthal,Jadhav,28,Male,Mumbai,Graduation,Science,2020,Yes,Harjai Technologies- Client: Essar Capital Ltd.,Data Entry Executive,Contract,Suitable


In [13]:
df_cols.columns

Index(['First_Name', 'Last_Name', 'Age', 'Gender', 'City',
       'Highest_Qualification', 'Stream', 'Year_Of_Completion',
       'Are_you_currently_working', 'Company_Name', 'Your_Designation',
       'Employment_Type', 'Suitability_Label'],
      dtype='object')

In [14]:


df_cols['Suitability_Label'] = df_cols['Suitability_Label'].replace({
    "Senior_Not_Suitable": "Not_Suitable",
    "MidSenior_Not_Suitable": "Not_Suitable"
})


In [15]:
df_cols['Suitability_Label'].value_counts()

Suitability_Label
Not_Suitable    100
Suitable         50
Name: count, dtype: int64

In [16]:
df_cols.to_csv("Employee.csv", index= False)